## Preprocessing

In [1]:
!pip install keras-tuner
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 3.7 MB/s eta 0:00:00


In [2]:
# Constants
APPLICATION_TYPE_THRES = 500
CLASSIFICATION_THRES = 1800
CHARITY_URL = "https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv"

In [3]:
application_df = pd.read_csv(CHARITY_URL)
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
trim_application_df = application_df.drop(['EIN', 'NAME'],axis=1)

In [5]:
# Determine the number of unique values in each column.
trim_application_df.nunique().sort_values()

STATUS                       2
SPECIAL_CONSIDERATIONS       2
IS_SUCCESSFUL                2
ORGANIZATION                 4
USE_CASE                     5
AFFILIATION                  6
INCOME_AMT                   9
APPLICATION_TYPE            17
CLASSIFICATION              71
ASK_AMT                   8747
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
trim_application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Choose a cutoff value (<500) and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
app_type_series = trim_application_df['APPLICATION_TYPE'].value_counts()
app_type_filter_series = app_type_series < APPLICATION_TYPE_THRES
# app_type_filter_series
app_type_series[app_type_filter_series].index  # instead of .values
application_types_to_replace = app_type_series[app_type_filter_series].index

In [8]:

# Replace in dataframe
for app in application_types_to_replace:
    trim_application_df['APPLICATION_TYPE'] = trim_application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
trim_application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
trim_application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
# You may find it helpful to look at CLASSIFICATION value counts >1
trim_application_df['CLASSIFICATION'].value_counts()[trim_application_df['CLASSIFICATION'].value_counts()>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [11]:
# Choose a cutoff value (<1800) and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_series = trim_application_df['CLASSIFICATION'].value_counts()
classifications_filter_series = classifications_series < CLASSIFICATION_THRES
# app_type_filter_series
classifications_series[classifications_filter_series].index  # instead of .values
classifications_to_replace = classifications_series[classifications_filter_series].index

In [12]:

# Replace in dataframe
for cls in classifications_to_replace:
    trim_application_df['CLASSIFICATION'] = trim_application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
trim_application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [13]:
# arguably cleaner code to splits off y first (based on class activity 20.2.8 instead of 19.2.3thru5...) so
# Split our preprocessed data into our features and target arrays
y = trim_application_df['IS_SUCCESSFUL']
X = trim_application_df.drop(columns='IS_SUCCESSFUL')
# y.head()
X.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590


In [14]:
# Convert categorical data to numeric with `pd.get_dummies` - arguably cleaner code to splits off y first so
X = pd.get_dummies(X)
X.head()

,STATUS,ASK_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [15]:

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# number_input_features = len(X_train[0]) # one neuron for each value in out input (column in X)
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=43, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 352       
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 403 (1.57 KB)
Trainable params: 403 (1.57 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=30)

Epoch 1/30
804/804 [==============================] - 7s 6ms/step - loss: 0.6202 - accuracy: 0.6635
Epoch 2/30
804/804 [==============================] - 3s 4ms/step - loss: 0.5667 - accuracy: 0.7193
Epoch 3/30
804/804 [==============================] - 4s 5ms/step - loss: 0.5599 - accuracy: 0.7285
Epoch 4/30
804/804 [==============================] - 8s 10ms/step - loss: 0.5573 - accuracy: 0.7295
Epoch 5/30
804/804 [==============================] - 5s 6ms/step - loss: 0.5553 - accuracy: 0.7308
Epoch 6/30
804/804 [==============================] - 5s 7ms/step - loss: 0.5537 - accuracy: 0.7315
Epoch 7/30
804/804 [==============================] - 6s 7ms/step - loss: 0.5531 - accuracy: 0.7322
Epoch 8/30
804/804 [==============================] - 2s 3ms/step - loss: 0.5520 - accuracy: 0.7322
Epoch 9/30
804/804 [==============================] - 2s 3ms/step - loss: 0.5515 - accuracy: 0.7326
Epoch 10/30
804/804 [==============================] - 3s 3ms/step - loss: 0.5512 - accuracy: 0.733

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5594 - accuracy: 0.7234 - 518ms/epoch - 2ms/step
Loss: 0.5594133138656616, Accuracy: 0.7233819365501404


In [21]:
# optimizing model via autotuning
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):  # hp stands for hyperparameters
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    # choice = choose from list  also could have more or different fxns

    # Allow kerastuner to decide number of neurons in first layer
    # Int = choose from an integer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=43))
# use the activation chosen above

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

  # always add the output layer
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model



In [22]:
# Import the kerastuner library
import keras_tuner as kt

# Create a `Hyperband()` tuner instance
# uses the create_model defined above
# hyperband give validation_accuracy  (is like a free R  data held out from training)
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [23]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))
# for each model that it creates train that model for 20 epochs
# this number of epochs * (hyperband_iterations+1) = total number of trials

Trial 60 Complete [00h 01m 24s]
val_accuracy: 0.7161515951156616

Best val_accuracy So Far: 0.7272303104400635
Total elapsed time: 00h 29m 33s


In [24]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 9,
 'num_layers': 1,
 'units_0': 3,
 'units_1': 1,
 'units_2': 3,
 'units_3': 5,
 'units_4': 9,
 'units_5': 3,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 2,
 'tuner/round': 0}

In [25]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5620 - accuracy: 0.7272 - 589ms/epoch - 2ms/step
Loss: 0.562040388584137, Accuracy: 0.7272303104400635


In [27]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5620 - accuracy: 0.7272 - 562ms/epoch - 2ms/step
Loss: 0.562040388584137, Accuracy: 0.7272303104400635
268/268 - 1s - loss: 0.5574 - accuracy: 0.7270 - 612ms/epoch - 2ms/step
Loss: 0.5574286580085754, Accuracy: 0.7269970774650574
268/268 - 1s - loss: 0.5659 - accuracy: 0.7269 - 590ms/epoch - 2ms/step
Loss: 0.5658947229385376, Accuracy: 0.7268804907798767


In [26]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE